Import libraries

In [1]:
%%capture
!pip install khayyam

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from khayyam import *
from datetime import date, datetime
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Q1:

Get Index data

In [3]:
data = pd.read_html('IndexData.xls')
arr = np.array(data).reshape(747, 3)
IndexData = pd.DataFrame(arr, columns=["name", "dateissue", "Value"])

In [4]:
IndexData.drop_duplicates(inplace=True)
IndexData = IndexData.sort_values(by='dateissue')
# IndexData['Value'] = IndexData['Value'].astype('int64')
IndexData['Value'] = IndexData['Value'].astype('float64')

In [5]:
# Tabdil be JalaliDate
IndexData['dateissue'] = IndexData['dateissue'].apply(lambda x: JalaliDate.strptime(str(x), '%Y%m%d'))

In [6]:
# Joda Kardane Year
IndexData['Year'] = IndexData['dateissue'].apply(lambda x: x.year)

In [7]:
# Tabdil be tarikhe miladi
IndexData['Date'] = IndexData['dateissue'].apply(lambda x: x.todate())
IndexData['Date'] = pd.to_datetime(IndexData['Date'])

In [8]:
IndexData.set_index('Date', inplace=True)

In [9]:
# Resample kardane haftegi baraye value
Index_returns = IndexData.resample('7D').mean()

In [10]:
Index_returns['IndexReturns'] = Index_returns['Value'].pct_change()

In [11]:
Index_97 = Index_returns[Index_returns['Year']==1397]
Index_98 = Index_returns[Index_returns['Year']==1398]
Index_99 = Index_returns[Index_returns['Year']==1399]

In [12]:
del Index_97['Year'], Index_97['Value']
del Index_98['Year'], Index_98['Value']
del Index_99['Year'], Index_99['Value']

Convert to numpy array

In [13]:
Index_97 = np.array(Index_97)
Index_98 = np.array(Index_98)
Index_99 = np.array(Index_99)

Risk free calculation

In [14]:
rf_97 = np.full(Index_97.shape, 0.00414533)
rf_98 = np.full(Index_98.shape, 0.00351234)
rf_99 = np.full(Index_98.shape, 0.00318804)
rf_99[26:] = 0.0036725

Get stocks data Function

In [15]:
def get_stock_data(FileAdress):
  # Baz kardane file
  df = pd.read_csv(FileAdress)
  # Tabdil kardane tarikhe miladi
  df['<DTYYYYMMDD>'] = pd.to_datetime(df['<DTYYYYMMDD>'], format='%Y%m%d')
  # Joda kardane 3 saal (97, 98, 99) az file
  df = df.iloc[:723,:]
  # Tabdil be tarikhe shamsi:
  df['تاریخ شمسی'] = df['<DTYYYYMMDD>'].apply(lambda x : JalaliDate(x).strftime('%Y %m %d'))
  # Sakhtane sotoone year:
  df['year'] = df['تاریخ شمسی'].str.split().str.get(0).astype('int64')
  # Resample kardane data be soorate haftegi:
  stock_data = df.resample('7D', on='<DTYYYYMMDD>').mean()
  # Mohasebe Return haftegi sahm:
  stock_data['StockReturns'] = stock_data['<CLOSE>'].pct_change()
  del stock_data['<CLOSE>']
  # Sakhtane 3 list Return baraye 3 sal (97, 98, 99):
  weekly_97 = stock_data[stock_data['year']==1397]
  weekly_98 = stock_data[stock_data['year']==1398]
  weekly_99 = stock_data[stock_data['year']==1399]
  del weekly_97['year']
  del weekly_98['year']
  del weekly_99['year']
  return np.array(weekly_97), np.array(weekly_98), np.array(weekly_99)

Gereftane etelaate saham:

In [16]:
Hkeshti_97, Hkeshti_98, Hkeshti_99 = get_stock_data('S IRI Marine Co.-a.csv')
Hamrah_97, Hamrah_98, Hamrah_99 = get_stock_data('Iran Mobil Tele-a.csv')
Fmeli_97, Fmeli_98, Fmeli_99 = get_stock_data('S I. N. C. Ind.-a.csv')
Shpna_97, Shpna_98, Shpna_99 = get_stock_data('S Isf. Oil Ref.Co.-a.csv')

Sakhtane Portfolio:

In [17]:
Portfo_97=np.mean([Hkeshti_97, Hamrah_97, Fmeli_97, Shpna_97], axis=0)
Portfo_98=np.mean([Hkeshti_98, Hamrah_98, Fmeli_98, Shpna_98], axis=0)
Portfo_99=np.mean([Hkeshti_99, Hamrah_99, Fmeli_99, Shpna_99], axis=0)

## CAPM calculation

In [18]:
def CAPM(stock_97, stock_98, stock_99):

  """
  Khorooji tabe 2 DataFrame ast.
  DataFrame aval etelaate capm beta bedoone intercept ra darad.
  DataFrame dovom etelaate capm beta ba intercept ra darad.
  """

  """Calculate beta without constant"""
  year=[97,98,99]
  beta=[]
  p_value=[]
  # year 97:
  model = sm.OLS(Index_97-rf_97, stock_97-rf_97, missing='drop').fit()
  beta.append(model.params[0])
  p_value.append(model.pvalues[0])
  # year 98:
  model = sm.OLS(Index_98-rf_98, stock_98-rf_98, missing='drop').fit()
  beta.append(model.params[0])
  p_value.append(model.pvalues[0])
  # year 99:
  model = sm.OLS(Index_99-rf_99, stock_99-rf_99, missing='drop').fit()
  beta.append(model.params[0])
  p_value.append(model.pvalues[0])
  # Making DataFrame: 
  df1 = pd.DataFrame({
      'year':year,
      'beta':beta,
      'P-value':p_value
  })

  """Calculate alpha and beta"""
  alpha = []
  alpha_p_value = []
  beta = []
  beta_p_value = []
  # year 97:
  w97_wc = sm.add_constant(stock_97-rf_97)
  model = sm.OLS(Index_97-rf_97, w97_wc, missing='drop').fit()
  alpha.append(model.params[0])
  alpha_p_value.append(model.pvalues[0])
  beta.append(model.params[1])
  beta_p_value.append(model.pvalues[1])
  # year 98:
  w98_wc = sm.add_constant(stock_98-rf_98)
  model = sm.OLS(Index_98-rf_98, w98_wc, missing='drop').fit()
  alpha.append(model.params[0])
  alpha_p_value.append(model.pvalues[0])
  beta.append(model.params[1])
  beta_p_value.append(model.pvalues[1])
  # year 99:
  w99_wc = sm.add_constant(stock_99-rf_99)
  model = sm.OLS(Index_99-rf_99, w99_wc, missing='drop').fit()
  alpha.append(model.params[0])
  alpha_p_value.append(model.pvalues[0])
  beta.append(model.params[1])
  beta_p_value.append(model.pvalues[1])
  # Making DataFrame: 
  df2 = pd.DataFrame({
      'year':year,
      'alpha':alpha,
      'alpha P-value':alpha_p_value,
      'beta':beta,
      'beta P-value':beta_p_value
  })
  return df1, df2

In [19]:
capm_hkeshti_1, capm_hkeshti_2 = CAPM(Hkeshti_97, Hkeshti_98, Hkeshti_99)
capm_hamrah_1, capm_hamrah_2 = CAPM(Hamrah_97, Hamrah_98, Hamrah_99)
capm_fmeli_1, capm_fmeli_2 = CAPM(Fmeli_97, Fmeli_98, Fmeli_99)
capm_shpna_1, capm_shpna_2 = CAPM(Shpna_97, Shpna_98, Shpna_99)
capm_portfo_1, capm_portfo_2 = CAPM(Portfo_97, Portfo_98, Portfo_99)

Print results:

In [20]:
def print_capm_results(stock_name, df1, df2):
  print('-'*20+stock_name+'-'*20)
  print('   CAPM results without constant:\n')
  print(df1)
  print('-'*30)
  print('\n   CAPM results with constant:\n')
  print(df2)
  print('\n')

In [21]:
print_capm_results('Hkeshti', capm_hkeshti_1, capm_hkeshti_2)
print_capm_results('Hamrah', capm_hamrah_1, capm_hamrah_2)
print_capm_results('Fmeli', capm_fmeli_1, capm_fmeli_2)
print_capm_results('Shpna', capm_shpna_1, capm_shpna_2)
print_capm_results('Portfolio', capm_portfo_1, capm_portfo_2)

--------------------Hkeshti--------------------
   CAPM results without constant:

   year      beta       P-value
0    97  0.453044  1.041864e-05
1    98  0.261405  5.177327e-08
2    99  0.470267  8.657546e-10
------------------------------

   CAPM results with constant:

   year     alpha  alpha P-value      beta  beta P-value
0    97  0.008026       0.084110  0.450486  8.417514e-06
1    98  0.012038       0.000431  0.220025  4.951598e-07
2    99  0.011114       0.094168  0.458411  1.394816e-09


--------------------Hamrah--------------------
   CAPM results without constant:

   year      beta       P-value
0    97  0.634844  2.743818e-03
1    98  0.350657  2.915668e-03
2    99  0.530868  8.631280e-10
------------------------------

   CAPM results with constant:

   year     alpha  alpha P-value      beta  beta P-value
0    97  0.008190       0.113609  0.630919  2.544302e-03
1    98  0.014550       0.000701  0.219868  4.542707e-02
2    99  0.011423       0.084891  0.517908  1.2763

## Part C: Risk

Market Risk

In [22]:
year = [97, 98, 99]
market_risks = [np.nanstd(Index_97), np.nanstd(Index_98), np.nanstd(Index_99)]

In [23]:
market_risk = pd.DataFrame({
    'Year':year,
    'Market Risk': market_risks
})

In [24]:
market_risk

,Year,Market Risk
0,97,0.039088
1,98,0.027906
2,99,0.066170


systematik risk

In [25]:
s_hkeshti = []
s_hamrah = []
s_fmeli = []
s_shpna = []
s_portfo = []

In [26]:
for i in range(3):
  s_hkeshti.append(market_risks[i] * capm_hkeshti_1.loc[i,'beta'])
  s_hamrah.append(market_risks[i] * capm_hamrah_1.loc[i,'beta'])
  s_fmeli.append(market_risks[i] * capm_fmeli_1.loc[i,'beta'])
  s_shpna.append(market_risks[i] * capm_shpna_1.loc[i,'beta'])
  s_portfo.append(market_risks[i] * capm_portfo_1.loc[i,'beta'])

In [27]:
systematic_risks = pd.DataFrame({
    'Year': year,
    'Hkeshti' : s_hkeshti,
    'Hamrah' : s_hamrah,
    'Fmeli' : s_fmeli,
    'Shpna' : s_shpna,
    'Portfolio' : s_portfo
})

In [28]:
print('\tSystematic Risks:\n')
print(systematic_risks)

	Systematic Risks:

   Year   Hkeshti    Hamrah     Fmeli     Shpna  Portfolio
0    97  0.017709  0.024815  0.020623  0.018789   0.037217
1    98  0.007295  0.009785  0.015435  0.011541   0.023253
2    99  0.031117  0.035127  0.035735  0.030357   0.046571


unsystematik risk

In [29]:
us_hkeshti = []
us_hamrah = []
us_fmeli = []
us_shpna = []
us_portfo = []

In [30]:
for i in range(3):
  us_hkeshti.append(
      (np.nanvar(get_stock_data('S IRI Marine Co.-a.csv')[i]) - s_hkeshti[i]**2)**(0.5)
  )
  us_hamrah.append(
      (np.nanvar(get_stock_data('S IRI Marine Co.-a.csv')[i]) - s_hamrah[i]**2)**(0.5)
  )
  us_fmeli.append(
      (np.nanvar(get_stock_data('S IRI Marine Co.-a.csv')[i]) - s_fmeli[i]**2)**(0.5)
  )
  us_shpna.append(
      (np.nanvar(get_stock_data('S IRI Marine Co.-a.csv')[i]) - s_shpna[i]**2)**(0.5)
  )

In [31]:
# unsystematic risk of portfolio:
us_portfo.append(
    np.sqrt(np.nanvar(Portfo_97) - s_portfo[0]**2)
)
us_portfo.append(
    np.sqrt(np.nanvar(Portfo_98) - s_portfo[1]**2)
)
us_portfo.append(
    np.sqrt(np.nanvar(Portfo_99) - s_portfo[2]**2)
)

In [32]:
unsystematic_risks = pd.DataFrame({
    'Year': year,
    'Hkeshti' : us_hkeshti,
    'Hamrah' : us_hamrah,
    'Fmeli' : us_fmeli,
    'Shpna' : us_shpna,
    'Portfolio' : us_portfo
})

In [33]:
print('\tUnSystematic Risks:\n')
print(unsystematic_risks)

	UnSystematic Risks:

   Year   Hkeshti    Hamrah     Fmeli     Shpna  Portfolio
0    97  0.047046  0.043717  0.045843  0.046625   0.006159
1    98  0.079862  0.079596  0.078696  0.079360   0.021285
2    99  0.099721  0.098381  0.098162  0.099955   0.075066
